In [1]:
import torch
import numpy as np

from pathlib import Path

In [13]:
# Configurations
model_name = "region3-lr8-migraphnet-59-60-61-62-63-15-5-wreg-hs-tm10"
area = "Region3LR4"

# Create correct paths
model_path = (Path("model-weights")
              / model_name 
              / "Predictions"
              / "predictions.pt")

out_fname = f"{model_name}.npy"
out_path = Path("../Data/") / area / "predictions" / out_fname
out_path.parent.mkdir(exist_ok=True, parents=True)

In [14]:
preds = torch.load(model_path)
if isinstance(preds, list):  # Make it compatible with old prediction writer
    preds = torch.cat(preds)

np.save(out_path, preds.float().flatten().squeeze().numpy())

In [4]:
preds.reshape((-1, 2)).min(dim=0)[0]

RuntimeError: shape '[-1, 2]' is invalid for input of size 10937493

# Convert normalized predictions

In [2]:
# Configurations
model_name = "region3-lr8-migraphnet-59-60-61-62-63-15-5-wreg-hs-dir-test"
area = "Region3LR4"
var = "Hs"
norm_years = "1959_1960_1961_1962_1963"

# Create correct paths
model_path = (Path("model-weights")
              / model_name 
              / "Predictions"
              / "predictions.pt")

out_fname = f"{model_name}.npy"
out_path = Path("../Data/") / area / "predictions" / out_fname
out_path.parent.mkdir(exist_ok=True, parents=True)

In [3]:
preds = torch.load(model_path)

In [4]:
# Load normalization parameters
mean, std = np.load(f"../Data/{area}/HR_6cpu/{var}/znorm_{norm_years}.npy")

In [5]:
preds = torch.load(model_path)
if isinstance(preds, list):  # Make it compatible with old prediction writer
    preds = torch.cat(preds)

preds = preds.float().flatten().squeeze().numpy()

np.save(out_path, preds * std + mean)

# Convert multi-region predictions

In [49]:
# Configurations
model_name = "region123-lr8-migraphnet-59-60-61-62-63-15-5-wreg-hs-dir"
area = "Region3LR4"
var = "Hs"
norm_years = "1959_1960_1961_1962_1963"

# Create correct paths
model_path = (Path("model-weights-curta")
              / model_name 
              / "Predictions"
              / "predictions.pt")

out_fname = f"{model_name}.npy"
out_path = Path("../Data/") / area / "predictions" / out_fname
out_path.parent.mkdir(exist_ok=True, parents=True)

In [87]:
preds = torch.load(model_path)

In [88]:
# Index for Region 1: 7*2338 + 8*364*2338
# Index for Region 2: 1*3747 + 8*364*3747 + 6*3747
# Index for Region 3: 2*2652 + 8*364*2652 + 5*2652
idx1 = 7*2338 + 8*364*2338
idx2 = idx1 + 1*3747 + 8*364*3747 + 6*3747
idx3 = idx2 + 2*2652 + 8*364*2652 + 5*2652

In [89]:
preds = torch.cat(preds)
pred1 = preds[:idx1]
pred2 = preds[idx1:idx2]
pred3 = preds[idx2:idx3]

In [90]:
# Load normalization parameters for the different regions
mean1, std1 = np.load(f"../Data/Region1LR4/HR_6cpu/{var}/znorm_{norm_years}.npy")
mean2, std2 = np.load(f"../Data/Region2LR4/HR_6cpu/{var}/znorm_{norm_years}.npy")
mean3, std3 = np.load(f"../Data/Region3LR4/HR_6cpu/{var}/znorm_{norm_years}.npy")

In [91]:
# Region 1
out_fname = f"{model_name}.npy"
out_path = Path("../Data/") / "Region1LR4" / "predictions" / out_fname
out_path.parent.mkdir(exist_ok=True, parents=True)

pred1 = pred1.float().flatten().squeeze().numpy()
np.save(out_path, pred1 * std1 + mean1)

# Region 2
out_fname = f"{model_name}.npy"
out_path = Path("../Data/") / "Region2LR4" / "predictions" / out_fname
out_path.parent.mkdir(exist_ok=True, parents=True)

pred2 = pred2.float().flatten().squeeze().numpy()
np.save(out_path, pred2 * std2 + mean2)

# Region 3
out_fname = f"{model_name}.npy"
out_path = Path("../Data/") / "Region3LR4" / "predictions" / out_fname
out_path.parent.mkdir(exist_ok=True, parents=True)

pred3 = pred3.float().flatten().squeeze().numpy()
np.save(out_path, pred3 * std3 + mean3)